The first step is to import the libraries and set the OpenAI API key and endpoint. You'll need to set the following environment variables:

- `AZURE_OPENAI_API_KEY` - Your OpenAI API key
- `AZURE_OPENAI_ENDPOINT` - Your OpenAI endpoint
- `AZURE_OPENAI_EMBEDDING_MODEL_DEPLOYMENT_NAME` - Your Azure OpenAI Embedding model deployment name
- `AZURE_COG_SEARCH_API_KEY` - Your Azure Cognitive Search API key
- `AZURE_COG_SEARCH_ENDPOINT` - Your Azure Cognitive Search endpoint

In [ ]:
import os
from dotenv import load_dotenv
import httpx
import openai
from openai.embeddings_utils import get_embedding

load_dotenv()

AZURE_COG_SEARCH_API_KEY = os.environ["AZURE_COG_SEARCH_API_KEY"]
AZURE_COG_SEARCH_SERVICE_NAME = os.environ["AZURE_COG_SEARCH_SERVICE_NAME"]
AZURE_COG_SEARCH_API_VERSION = "2023-07-01-Preview"
AZURE_COG_SEARCH_INDEX_NAME = "product-catalog-index"


openai.api_type = "azure"
openai.api_key = os.environ["AZURE_OPENAI_API_KEY"]
openai.api_base = os.environ["AZURE_OPENAI_ENDPOINT"]
openai.api_version = "2023-07-01-preview"

OPENAI_EMBEDDING_ENGINE = "text-embedding-ada-002"

OPENAI_EMBEDDING_DEPLOYMENT_NAME = os.environ[
    "AZURE_OPENAI_EMBEDDING_MODEL_DEPLOYMENT_NAME"
]

In [ ]:
# pretty print the list of results
def pretty_print(results):

    for result in results:
        print(f"{result['title']} ({result['category']}) ({result['@search.score']})")
        print(f"{result['content'][:200]}...")
        print()

Get the embedding for the query. The query must be embedded before it can be used in a search query and it must be embedded using the same model that was used to embed the documents.

In [ ]:
query = "getting started with azure search"

query_embeddings = get_embedding(query, OPENAI_EMBEDDING_ENGINE)

In [ ]:
# Query the Azure Cognitive Search index using the REST API

request = {
    "vector": {
        "value": query_embeddings,
        "fields": "contentVector",
        "k": 5,
    },
    "select": "title, content, category",
}

url = (
    f"https://{AZURE_COG_SEARCH_SERVICE_NAME}.search.windows.net/indexes/{AZURE_COG_SEARCH_INDEX_NAME}"
    f"/docs/search?api-version={AZURE_COG_SEARCH_API_VERSION}"
)

headers = {
    "Content-Type": "application/json",
    "api-key": AZURE_COG_SEARCH_API_KEY,
}

response = httpx.post(url, json=request, headers=headers)
response.raise_for_status()

results = response.json()["value"]

pretty_print(results)

In [ ]:
# Constrain results using a filter query

request = {
    "vector": {
        "value": query_embeddings,
        "fields": "contentVector",
        "k": 5,
    },
    "select": "title, content, category",
    "filter": "category eq 'Analytics'"
}

url = (
    f"https://{AZURE_COG_SEARCH_SERVICE_NAME}.search.windows.net/indexes/{AZURE_COG_SEARCH_INDEX_NAME}"
    f"/docs/search?api-version={AZURE_COG_SEARCH_API_VERSION}"
)

headers = {
    "Content-Type": "application/json",
    "api-key": AZURE_COG_SEARCH_API_KEY,
}

response = httpx.post(url, json=request, headers=headers)
response.raise_for_status()

results = response.json()["value"]

pretty_print(results)